In [22]:
# import packages
import time, math, os
from tqdm import tqdm
import gc
import pickle
import random
from datetime import datetime
from operator import itemgetter
import numpy as np
import pandas as pd
import warnings
from collections import defaultdict
import collections
warnings.filterwarnings('ignore')

In [23]:
data_dir = "/users/liupenghua/Documents/docs/竞赛/入门-推荐系统-新闻推荐"
save_dir = "/Users/liupenghua/Documents/docs/竞赛/入门-推荐系统-新闻推荐/checkpoints"

In [24]:
# 'article_id', 'category_id', 'created_at_ts', 'words_count'
articles = pd.read_csv(data_dir + "/articles.csv")
print(articles.columns)
articles.head()

Index(['article_id', 'category_id', 'created_at_ts', 'words_count'], dtype='object')


,article_id,category_id,created_at_ts,words_count
0,0,0,1513144419000,168
1,1,1,1405341936000,189
2,2,1,1408667706000,250
3,3,1,1408468313000,230
4,4,1,1407071171000,162


In [25]:
# 'article_id', 'emb_0', 'emb_1', ..., 'emb_249'
articles_emb = pd.read_csv(data_dir + "/articles_emb.csv")
print(articles_emb.columns)
articles_emb.head()

Index(['article_id', 'emb_0', 'emb_1', 'emb_2', 'emb_3', 'emb_4', 'emb_5',
       'emb_6', 'emb_7', 'emb_8',
       ...
       'emb_240', 'emb_241', 'emb_242', 'emb_243', 'emb_244', 'emb_245',
       'emb_246', 'emb_247', 'emb_248', 'emb_249'],
      dtype='object', length=251)


,article_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_240,emb_241,emb_242,emb_243,emb_244,emb_245,emb_246,emb_247,emb_248,emb_249
0,0,-0.161183,-0.957233,-0.137944,0.050855,0.830055,0.901365,-0.335148,-0.559561,-0.500603,...,0.321248,0.313999,0.636412,0.169179,0.540524,-0.813182,0.286870,-0.231686,0.597416,0.409623
1,1,-0.523216,-0.974058,0.738608,0.155234,0.626294,0.485297,-0.715657,-0.897996,-0.359747,...,-0.487843,0.823124,0.412688,-0.338654,0.320786,0.588643,-0.594137,0.182828,0.397090,-0.834364
2,2,-0.619619,-0.972960,-0.207360,-0.128861,0.044748,-0.387535,-0.730477,-0.066126,-0.754899,...,0.454756,0.473184,0.377866,-0.863887,-0.383365,0.137721,-0.810877,-0.447580,0.805932,-0.285284
3,3,-0.740843,-0.975749,0.391698,0.641738,-0.268645,0.191745,-0.825593,-0.710591,-0.040099,...,0.271535,0.036040,0.480029,-0.763173,0.022627,0.565165,-0.910286,-0.537838,0.243541,-0.885329
4,4,-0.279052,-0.972315,0.685374,0.113056,0.238315,0.271913,-0.568816,0.341194,-0.600554,...,0.238286,0.809268,0.427521,-0.615932,-0.503697,0.614450,-0.917760,-0.424061,0.185484,-0.580292


In [26]:
# 'user_id', 'click_article_id', 'click_timestamp', 'click_environment', 
# 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 
# 'click_referrer_type'
train_clicks = pd.read_csv(data_dir + '/train_click_log.csv')
print(train_clicks.columns)
train_clicks.head()


Index(['user_id', 'click_article_id', 'click_timestamp', 'click_environment',
       'click_deviceGroup', 'click_os', 'click_country', 'click_region',
       'click_referrer_type'],
      dtype='object')


,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5


In [27]:
# 'user_id', 'click_article_id', 'click_timestamp', 'click_environment',
# 'click_deviceGroup', 'click_os', 'click_country', 'click_region',
# 'click_referrer_type'
test_clicks = pd.read_csv(data_dir + '/testA_click_log.csv')
print(test_clicks.columns)
test_clicks.head()

Index(['user_id', 'click_article_id', 'click_timestamp', 'click_environment',
       'click_deviceGroup', 'click_os', 'click_country', 'click_region',
       'click_referrer_type'],
      dtype='object')


,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,249999,160974,1506959142820,4,1,17,1,13,2
1,249999,160417,1506959172820,4,1,17,1,13,2
2,249998,160974,1506959056066,4,1,12,1,13,2
3,249998,202557,1506959086066,4,1,12,1,13,2
4,249997,183665,1506959088613,4,1,17,1,15,5


## 数据探索分析

#### df 节省内存函数

In [28]:
def reduce_mem(df):
    """节省 df 的空间存储, 通过对各个数值列的数据分布，确定 int 和 float 的精度
    """
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df

#### 采样或加载全部点击数据

In [29]:
# debug模式：从训练集中划出一部分数据来调试代码
def get_all_click_sample(data_path, sample_nums=10000):
    """从点击明细数据中采样一部分用户的数据, 用于debug

    Args:
        data_path (str): 数据存储路径
        sample_nums (int, optional): 待采样的用户数. Defaults to 10000.

    Returns:
        all_click(<pd.DataFrame>): 采样后的数据集
    """
    all_click = pd.read_csv(data_path + '/train_click_log.csv')
    all_user_ids = all_click.user_id.unique()

    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False) 
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    
    # 按 user_id, click_article_id, click_timestamp 去重
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click


def get_all_click_df(data_path='./data_raw', offline=True):
    """ 读取点击数据，区分本地和线上模式：
            本地: 只使用训练集, 测试集用于模型校准
            线上: 将测试集合并到训练集，联合训练

    Args:
        data_path (str, optional): 数据存储路径. Defaults to './data_raw'.
        offline (bool, optional): 是否离线/本地. Defaults to True.

    Returns:
        all_click(<pd.DataFrame>): 训练数据集
    """
    if offline:
        all_click = pd.read_csv(data_path + '/train_click_log.csv')
    else:
        trn_click = pd.read_csv(data_path + '/train_click_log.csv')
        tst_click = pd.read_csv(data_path + '/testA_click_log.csv')
        # 合并 train & testA, 作为联合的训练集
        all_click = trn_click.append(tst_click)
    # 按 user_id, click_article_id, click_timestamp 去重
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

In [30]:
# 全量训练集
all_click_df = get_all_click_df(data_dir, offline=False)

#### 构造 用户x文章x时间 点击字典

In [31]:
def get_user_item_time(click_df):
    """根据点击时间获取用户的点击文章序列   {user1: [(item1, time1), (item2, time2)..]...}

    Args:
        click_df (<pd.DataFrame>): user x article x timestamp 事实明细

    Returns:
        user_item_time_dict(Dict[int, List[Tuple[int, int]]]): 字典, 主 key 为 user_id, value 为 (article_id, timestamp) 的顺序列表
    """
    
    click_df = click_df.sort_values('click_timestamp')
    
    def make_item_time_pair(df):
        # 构造 [article_id, timestamp] 组合
        return list(zip(df['click_article_id'], df['click_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'].apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict

In [32]:
def get_item_topk_click(click_df, k):
    """ 获取近期点击最多的文章
    """
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click

#### itemcf的物品相似度计算

In [33]:
def itemcf_sim(df):
    """利用 item 协同过滤, 计算文章与文章之间的相似性矩阵计算 (在多路召回部分会加上关联规则的召回策略)
    
    Args:
        df (<pd.DataFrame>): 文章创建时间的字典

    Returns:
        i2i_sim_ (Dict[int, Dict[int, float]]): 文章与文章的相似性矩阵, {'article_1': {'article_2': 0.5, 'article_3': 0.3}, ...}
    """
    
    user_item_time_dict = get_user_item_time(df)
    
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int) # item_id: clicked_cnt
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        # 在基于商品的协同过滤优化的时候可以考虑时间因素
        for i, i_click_time in item_time_list:
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for j, j_click_time in item_time_list:
                if(i == j):
                    continue
                i2i_sim[i].setdefault(j, 0)
                
                i2i_sim[i][j] += 1 / math.log(len(item_time_list) + 1)
                
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim_, open(save_dir + '/itemcf_i2i_sim.pkl', 'wb'))
    
    return i2i_sim_

In [35]:
i2i_sim = itemcf_sim(all_click_df)

100%|██████████| 250000/250000 [00:14<00:00, 17497.33it/s]


In [36]:
# 查看某个特定的商品的 i2i 向量
i2i_sim[30760]

{157507: 0.008340645572844295,
 211442: 0.04731573623276438,
 79851: 0.025172345057546546,
 111338: 0.021484016294237133,
 62465: 0.002126724769182523,
 70986: 0.024590703055713432,
 234481: 0.04166324864214384,
 59423: 0.0021200744185940197,
 59146: 0.017194378334467124,
 58499: 0.004028005316366542,
 59225: 0.002733927289942795,
 95982: 0.00911878486954102,
 315104: 0.0074224653875016025,
 36162: 0.07042872697564427,
 331116: 0.012282708329128008,
 352121: 0.0028608178298358536,
 33297: 0.0074242004818470585,
 324426: 0.01831048794869324,
 211722: 0.004346429679363331,
 362914: 0.004583159729938785,
 209122: 0.021537022932166482,
 234308: 0.005986537866498491,
 50644: 0.0463042434725357,
 206785: 0.0014434688999688048,
 156279: 0.03343746200437497,
 16346: 0.015431384347569613,
 30955: 0.010326855076256455,
 206415: 0.006130938515323134,
 321350: 0.0041299962748346086,
 277107: 0.014578333455982473,
 283009: 0.003613508025672678,
 57442: 0.0019621676693927035,
 70758: 0.0152127028631

#### itemcf 的文章推荐

In [37]:
# 基于商品的召回i2i
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click):
    """ 基于 item 协同过滤的召回

    Args:
        user_id (int): 用户 id
        user_item_time_dict (Dict[int, List[Tuple[int, int]]):根据点击时间获取用户的点击文章序列   {user1: [(item1, time1), (item2, time2)..]...}
        i2i_sim (Dict[int, Dict[int, float]]): i2i 相似度矩阵
        sim_item_topk (int): top k 中的 k, 用于选取最相似的 k 个 item
        recall_item_num (int): 召回的数量
        item_topk_click (List[int]): 点击次数最多的文章列表，用户召回补全   
    
    Returns:
        : 召回的文章列表 {item1:score1, item2: score2...}
    """
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id]
    user_hist_items_ = {user_id for user_id, _ in user_hist_items}
    
    item_rank = {}
    # 遍历历史点击的 item 序列，分别计算每个 item 的 topk 相似的 item，将相似度分数累加
    for loc, (i, click_time) in enumerate(user_hist_items):
        # 对 item-i 的交互商品，按照相似度取 `sim_item_topk` (去除掉该 user 历史点击过的商品)
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items_:
                continue
                
            item_rank.setdefault(j, 0)
            item_rank[j] +=  wij
    
    # 不足10个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 赋值负数，使其排序在后面
            if len(item_rank) == recall_item_num:
                break
    
    # 对召回的商品按分数倒排，卡前 recall_item_num 个
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
        
    return item_rank

#### 给每个用户根据物品的协同过滤推荐文章

In [38]:
# 定义
user_recall_items_dict = collections.defaultdict(dict)

# 获取 用户 - 文章 - 点击时间的字典
user_item_time_dict = get_user_item_time(all_click_df)

# 去取文章相似度
i2i_sim = pickle.load(open(save_dir + '/itemcf_i2i_sim.pkl', 'rb'))

# 相似文章的数量, 作为候选
sim_item_topk = 10

# 召回文章数量，实际召回的文章数量
recall_item_num = 5

# 用户热度补全
item_topk_click = get_item_topk_click(all_click_df, k=50)

# all_click_df = get_all_click_sample(data_dir, sample_nums=10)
for user in tqdm(all_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, 
                                                        sim_item_topk, recall_item_num, item_topk_click)

100%|██████████| 250000/250000 [22:19<00:00, 186.69it/s]


#### 召回字典转换成df

In [39]:
# 将字典的形式转换成df
user_item_score_list = []

for user, items in tqdm(user_recall_items_dict.items()):
    for item, score in items:
        user_item_score_list.append([user, item, score])

recall_df = pd.DataFrame(user_item_score_list, columns=['user_id', 'click_article_id', 'pred_score'])

100%|██████████| 250000/250000 [00:02<00:00, 120228.61it/s]


#### 生成提交文件

In [40]:
# 生成提交文件
def submit(recall_df, topk=5, model_name=None):
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 判断是不是每个用户都有5篇文章及以上
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk
    
    del recall_df['pred_score']
    submit = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()
    
    submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    # 按照提交格式定义列名
    submit = submit.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2', 
                                                  3: 'article_3', 4: 'article_4', 5: 'article_5'})
    
    save_name = save_dir + "/" + model_name + '_' + datetime.today().strftime('%m-%d') + '.csv'
    submit.to_csv(save_name, index=False, header=True)

In [41]:
# 获取测试集
tst_click = pd.read_csv(data_dir + '/testA_click_log.csv')
tst_users = tst_click['user_id'].unique()

# 从所有的召回数据中将测试集中的用户选出来
tst_recall = recall_df[recall_df['user_id'].isin(tst_users)]

# 生成提交文件
submit(tst_recall, topk=5, model_name='itemcf_baseline')

#### 最终比赛成绩

上述结果提交后，线上结果为 0.1026，距离前100名（0.2837）还有很大的提升空间。